# 🤖 **RPA** - 📧 Mailing Automático de Reporte de Tipos de Cambio 💵

## 📚 Instalación de librerías

Para instalar correctamente los paquetes necesarios, puedes utilizar el siguiente código quitando los numerales:

In [ ]:
# import sys
# !conda install --yes --prefix {sys.prefix} pandas
# !conda install --yes --prefix {sys.prefix} numpy
# !conda install --yes --prefix {sys.prefix} matplotlib
# !conda install --yes --prefix {sys.prefix} seaborn
# !conda install --yes --prefix {sys.prefix} selenium
# pip install webdriver-manager

Por otra parte, también vas a necesitar el chromedriver, o el driver necesario para tu navegador web. Este Software se descarga simplemente de la web oficial, https://chromedriver.chromium.org/. Una vez descargado, se ubica en la misma ruta que el script como es en el caso del presente notebook, o dentro del script puede informar la ruta de la siguiente manera, obviamente, quitándo los numerales que convierten las líneas de código en comentarios:

In [ ]:
# ruta_webdriver = r"C:\ruta\webdriver"
# driver = webdriver.Chrome(ruta_webdriver, options=chrome_options)

## 📚 Importación de librerías

In [ ]:
# Manejo del tiempo para cronometrar el programa y manejar datos tipo fecha
import time
import datetime as dt
from datetime import datetime
comienzo = time.perf_counter()

# Aplicar formato fecha del español
import locale

# Manejar datos y cálculos
import pandas as pd
import numpy as np

# Manejar APIs
import json
import requests

# Webscrapping
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Crear gráficos
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.ticker as ticker
from matplotlib.ticker import FormatStrFormatter, FuncFormatter, MultipleLocator, AutoMinorLocator
import seaborn as sns

# No mostrar contraseña
from getpass import getpass

# Personalización del mail
import mimetypes
from email.message import EmailMessage
from email.mime.image import MIMEImage
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Seguridad para el mailing automático
import smtplib, ssl

# Personalizar df como tabla html
from tabulate import tabulate

In [ ]:
# Función para eliminar el signo peso
def eliminar_peso(x):
    x = x.lstrip('$')
    return float(x)

# Función para reemplazar la coma de un string y convertirlo a número
def reemplazar_coma(x):
    x = x.replace(',', '.')
    return float(x)

In [ ]:
# Fuentes,usaremos las variables para ingresar a las webs y para citar las fuentes
web_dolarhoy = "https://dolarhoy.com/"
web_bna = "https://www.bna.com.ar/Personas"
web_ambito_mep = "https://www.ambito.com/contenidos/dolar-mep.html"
web_ambito_riesgo_pais = "https://www.ambito.com/contenidos/riesgo-pais-historico.html"
web_fed = "https://www.federalreserve.gov/releases/h15/"
web_bcra = "https://www.bcra.gob.ar/PublicacionesEstadisticas/Principales_variables.asp"
web_perfil= "https://www.perfil.com/seccion/economia"

## 📝 Importación de Excel acumulativo 

In [ ]:
# # Data to insert as a dictionary
# new_data = {
#     "Fecha": ["29/09/23"],
#     "Solidario": [639.63],
#     "TCC_Blue": [795.00],
#     "TCV_Blue": [800.00],
#     "TCV_MEP": [710.83],
#     "TCC_Billete": [347.50],
#     "TCV_Billete": [365.50],
#     "TCC_Divisas": [348.95],
#     "TCV_Divisas": [349.95],
#     "TCC_Euro": [813.00],
#     "TCV_Euro": [824.00],
#     "fed_tea": [5.33],
#     "bcra_tea": [209.45],
#     "riesgo_pais": [2486.00]
# }

# # Convert the new data dictionary into a DataFrame
# new_data_df = pd.DataFrame(new_data)

# # Append the new data to your existing DataFrame
# data_df = pd.concat([df, new_data_df], ignore_index=True)

# # Convert the "Fecha" column to datetime with the correct format
# data_df['Fecha'] = pd.to_datetime(data_df['Fecha'], format='%d/%m/%y')

# # Now, sort the DataFrame by the "Fecha" column in descending order
# data_df.sort_values(by="Fecha", ascending=False, inplace=True)

# # Reset the index
# data_df.reset_index(drop=True, inplace=True)

# data_df['Fecha'] = data_df['Fecha'].dt.strftime("%d/%m/%y")

# # Now, data_df is sorted by the "Fecha" column in descending order with a reset index
# data_df

In [ ]:
ruta_bbdd = r'Seguimiento Dólar Blue.csv'
df = pd.read_csv(ruta_bbdd, index_col=0)
df

## 🤖 Ingreso a la web y toma de datos 

In [ ]:
# Ingreso a la web dolarhoy
driver = webdriver.Chrome("chromedriver", options=chrome_options)
driver.get(web_dolarhoy)

# Toma de datos mediante xpath
solidario = driver.find_element(
    "xpath",
    '//*[@id="home_0"]/div[2]/section/div/div/div/div[1]/div/div[2]/div[6]/div/div[2]/div[2]')
blue_tcc = driver.find_element(
    "xpath",
    '//*[@id="home_0"]/div[2]/section/div/div/div/div[1]/div/div[1]/div/div[1]/div[1]/div[2]')
blue_tcv = driver.find_element(
    "xpath",
    '//*[@id="home_0"]/div[2]/section/div/div/div/div[1]/div/div[1]/div/div[1]/div[2]/div[2]')

# Creación de fila nueva, la columna fecha toma la fecha de hoy y la convierte en serie de tiempo
# - dt.timedelta(days=1) para restar un día
dolar_hoy = pd.DataFrame(
    {"Fecha":dt.datetime.today(),
     "Solidario":solidario.text,
     "TCC_Blue":blue_tcc.text,
     "TCV_Blue":blue_tcv.text},
    index=[0])

dolar_hoy["Fecha"] = pd.to_datetime(dolar_hoy["Fecha"]).dt.strftime("%d/%m/%y")

# Eliminación de $
dolar_hoy.Solidario = dolar_hoy.Solidario.apply(eliminar_peso)
dolar_hoy.TCC_Blue = dolar_hoy.TCC_Blue.apply(eliminar_peso)
dolar_hoy.TCV_Blue = dolar_hoy.TCV_Blue.apply(eliminar_peso)

display(dolar_hoy)

driver.close()

In [ ]:
#Ingreso a la web Banco Nación
driver = webdriver.Chrome('chromedriver', options=chrome_options)
driver.get(web_bna)

billete_tcc = driver.find_element(
    "xpath",
    '//*[@id="billetes"]/table/tbody/tr[1]/td[2]')
billete_tcv = driver.find_element(
    "xpath",
    '//*[@id="billetes"]/table/tbody/tr[1]/td[3]')

bna = pd.DataFrame({
    "TCC_Billete":[reemplazar_coma(billete_tcc.text)],
    "TCV_Billete":[reemplazar_coma(billete_tcv.text)]},
    index=[0])

#Cliquear botón que habilita el tipo de cambio divisas
driver.find_element(
    "xpath",
    '//*[@id="rightHome"]/div[1]/div/ul/li[2]/a').click()

#Esperar dos segundos para que aparezcan los elementos, wait.until no sirvió
time.sleep(2)

bna_tcc_divisas = driver.find_element(
    "xpath",
    '//*[@id="divisas"]/table/tbody/tr[1]/td[2]')
bna_tcv_divisas = driver.find_element(
    "xpath",
    '//*[@id="divisas"]/table/tbody/tr[1]/td[3]')
bna_divisas = pd.DataFrame(
    {"TCC_Divisas":float(bna_tcc_divisas.text),
    "TCV_Divisas":float(bna_tcv_divisas.text)},
    index=[0])

bna = pd.concat([bna,bna_divisas], axis = 1)

display(bna)

driver.close()

In [ ]:
driver = webdriver.Chrome('chromedriver', options=chrome_options)
driver.get(web_perfil)

tcc_euro = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located(
        ("xpath", '//*[@id="app-body"]/main/div[1]/div/div/div[24]')
    )
)

tcv_euro =  WebDriverWait(driver, 10).until(
    EC.presence_of_element_located(
        ("xpath", '//*[@id="app-body"]/main/div[1]/div/div/div[23]')
    )
)

euro = pd.DataFrame(
    {"TCC_Euro": [float(tcc_euro.text.replace(".","").replace(",","."))],
    "TCV_Euro": [float(tcv_euro.text.replace(".","").replace(",","."))]},
    index=[0]
)

display(euro)

driver.close()

In [ ]:
driver = webdriver.Chrome('chromedriver', options=chrome_options)
driver.get(web_ambito_mep)

time.sleep(2)

mep = driver.find_element(
    "xpath",
    '/html/body/main/div/div[1]/div[1]/div/div[2]/div[1]/div/div/div[2]/span[1]')

ambito = pd.DataFrame(
    {"TCV_MEP":[reemplazar_coma(mep.text)]},
    index=[0])

driver.get(web_ambito_riesgo_pais)
time.sleep(2)

riesgo_pais = driver.find_element(
    "xpath",
    '/html/body/main/div/div[1]/div[10]/table/tbody/tr[1]/td[2]')

riesgo_pais = pd.DataFrame(
    {"riesgo_pais":[reemplazar_coma(riesgo_pais.text)]},
    index=[0])

ambito = pd.concat([ambito,riesgo_pais],axis = 1)

display(ambito)

driver.close()

In [ ]:
# Ingresamos a la web de la FED para obtener la tasa de interés de EEUU
driver = webdriver.Chrome('chromedriver', options=chrome_options)
driver.get(web_fed)

fed_tea = driver.find_element(
    "xpath",
    '/html/body/div[3]/div[6]/table/tbody/tr[1]/td[5]')
time.sleep(2)

fed_tea = pd.DataFrame(
{"fed_tea":[reemplazar_coma(fed_tea.text)]},
index=[0])

time.sleep(2)

driver.get(web_bcra)
time.sleep(2)

# Ingresamos a la web del BCRA para obtener la tasa de interés y la inflación de Argentina 
driver = webdriver.Chrome('chromedriver', options=chrome_options)
driver.get(web_bcra)

bcra_tea = driver.find_element(
    "xpath",
    '/html/body/div/div[2]/div/div/div/div/table/tbody/tr[8]/td[3]')
time.sleep(2)

bcra_tea = pd.DataFrame(
    {"bcra_tea":[reemplazar_coma(bcra_tea.text)]},
    index=[0])

# Concatenamos las tasas para crear las columnas a agregar
tasas = pd.concat([fed_tea,bcra_tea], axis=1)

driver.close()

# Imprimimos ambos dataframes con dos renglones de diferencia
display(tasas)

In [ ]:
# Obtenemos la inflación mucho más rápido mediante api que mediante scraping
base_url = "https://api.estadisticasbcra.com/"
token = string_de_tu_token_gratuito
consulta = "inflacion_mensual_oficial"
url = base_url + consulta

headers = {"Authorization": f"BEARER {token}"}
response = requests.get(url, headers=headers).json()
inflacion_df = pd.json_normalize(response)

inflacion_df.columns = ["Fecha", "Inflación"]
inflacion_df["Fecha"] = pd.to_datetime(inflacion_df["Fecha"], format="%Y-%m-%d")

inflacion_df

In [ ]:
#Editamos el formato de las fechas, no tiene sentido tener el último día del mes
inflacion_df.Fecha = pd.to_datetime(inflacion_df.Fecha).dt.strftime('%m/%y')

#Tomamos sólo los últimos 96 meses (8 años)
inflacion_df = inflacion_df.tail(96).copy()
cantidad_filas = inflacion_df.shape[0]

#A la columna inflación la dividimos por 100 para que sea %, le sumamos 1 para poder multiplicar
#Le aplicamos cumprod (multiplica acumuladamente) y le restamos el uno que sumamos para cada fila
#Lo multiplicamos por 100 para que coincida con el formato del eje Y de antes
inflacion_df["Inflación Agregada"] = ((inflacion_df["Inflación"]/100+1).cumprod()-1)*100
inflacion_df

In [ ]:
#nueva fila final con todos los datos tomados
fila_nueva = [dolar_hoy, ambito, bna, euro, tasas]
fila_nueva = pd.concat(fila_nueva, axis=1)
fila_nueva

## 🖥️ Concatenado de la nueva fila en el DataFrame acumulado 

In [ ]:
# Concatenado de la fila nueva
df = pd.concat([
    fila_nueva,
    df.loc[:]
]).reset_index(drop=True)

In [ ]:
# Bajo paridad de tasas de interés de Irving Fisher a 3 meses
bcra_tea_numerador = 1 + (float(df["bcra_tea"].iloc[0])/100)
fed_tea_denominador = 1 + (float(df["fed_tea"].iloc[0])/100)
division = bcra_tea_numerador / fed_tea_denominador
fwd_oficial = float(df["TCV_Billete"].iloc[0]) * division
fwd_blue = float(df["TCV_Blue"].iloc[0]) * division

In [ ]:
# Crear columnas de cálculo de brechas y variación del día
df["Solidario / TCV Blue"] = df["Solidario"]/df["TCV_Blue"]-1
df["TCV MEP / TCV Blue"] = df["TCV_MEP"]/df["TCV_Blue"]-1
df["TCV Euro / TCC Blue %"] = df["TCV_Euro"]/df["TCV_Blue"]-1
df["Variación Solidario"] = df["Solidario"].pct_change(periods=-1)
df["Variación Solidario"].fillna(0, inplace=True)
df["Variación TCV Blue"] = df["TCV_Blue"].pct_change(periods=-1)
df["Variación TCV Blue"].fillna(0, inplace=True)
df["Variación TCV Euro"] = df["TCV_Euro"].pct_change(periods=-1)
df["Variación TCV Euro"].fillna(0, inplace=True)
df

## 💾 Exportar a CSV

In [ ]:
# df.to_csv(ruta_bbdd)

## 📈 Gráficos para adjuntar en el mail

In [ ]:
# Tomamos las primeras filas que querramos mostrar y las primeras 14 columnas, es decir, sin variaciones
cotizaciones_a_mostrar = 25
# Copia del dataframe, para no reemplazarlo
data = df.copy()
# Damos vuelta el dataframe sólo para los gráficos
# Si no lo damos vuelta, los gráficos van a empzar de hoy hacia atrás
data = data.iloc[::-1]
data

In [ ]:
# Creo un subplot de 3 gráficos verticales
fig, ax = plt.subplots(
    3, 1,
    figsize=(10, 14),
    sharex=False
)

# Estilo de gráfico, con ticks en X e Y
sns.set(style="ticks")

titulos = [
    "Cotizaciones Paralelas",
    "Cotizaciones Oficiales",
    "Evolución del Riesgo País",
]

ax0_palette = sns.color_palette("dark")

ax0_columns = [
    "TCV_Euro",
    "TCC_Blue",
    "TCV_Blue",
    "TCV_MEP",
    "Solidario"
]
ax0_labels = [
    "TCV Euro Blue",
    "TCC Blue",
    "TCV Blue",
    "MEP",
    "Solidario"
]

# Títulos general y para cada gráfico
fig.suptitle(
    f"Gráficos tipos de cambio y riesgo país - últimas {cotizaciones_a_mostrar} cotizaciones",
    fontweight="bold",
    fontsize=18
)
for i, titulo in enumerate(titulos):
    ax[i].set_title(
        titulo,
        fontweight="bold",
        fontsize=12
    )

for i, column in enumerate(ax0_columns):
    sns.lineplot(
        x="Fecha",
        y=column,
        data=data.tail(cotizaciones_a_mostrar),
        label=ax0_labels[i],
        color=ax0_palette[i],
        marker="o",
        ax=ax[0],
    )

# Gráfico de cotizaciones oficiales
ax1_palette = sns.color_palette("bright")
ax1_columns = [
    "TCC_Divisas",
    "TCV_Divisas",
    "TCC_Billete",
    "TCV_Billete"
]
ax1_labels = [
    "TCC Divisas",
    "TCV Divisas",
    "TCC Billete",
    "TCV Billete"
]
for i, column in enumerate(ax1_columns):
    sns.lineplot(
        x="Fecha",
        y=column,
        data=data.tail(cotizaciones_a_mostrar),
        label=ax1_labels[i],
        color=ax1_palette[i],
        marker="D",
        ax=ax[1],
    )

# Gráfico de barras del riesgo país
sns.barplot(
    x="Fecha",
    y="riesgo_pais",
    data=data.tail(cotizaciones_a_mostrar),
    label="Riesgo País",
    color="darkred",
    ax=ax[2],
)

# Agregamos etiquetas de datos a las barras
for i, bar in enumerate(ax[2].containers[0].get_children()[-len(data.riesgo_pais) :]):
    height = bar.get_height()
    ax[2].annotate(
        f"{height:,.0f}",
        xy=(bar.get_x() + bar.get_width() / 2, height),
        xytext=(0, 3),
        textcoords="offset points",
        ha="center",
        va="bottom",
        fontproperties=fm.FontProperties(weight="bold", size=9),
    )

# Consolidate the formatting of x-axis
for axis in ax:
    axis.set_xlabel("")
    axis.xaxis.set_major_locator(MultipleLocator(2))
    axis.xaxis.set_minor_locator(MultipleLocator(1))

# y separador de miles, por si la inflación se va demasiado, ya estoy adelantado
for i in range(2):
    ax[i].yaxis.set_major_formatter(FuncFormatter("{:,.2f}".format))

# Títulos y ticks eje Y
y_labels = ["ARS / USD", "ARS / USD", "Puntos Base"]
y_major_locators = [
    MultipleLocator(20),
    MultipleLocator(5),
    MultipleLocator(500)
]
y_minor_locators = [
    MultipleLocator(5),
    MultipleLocator(1),
    MultipleLocator(100)
]
for i in range(3):
    ax[i].set_ylabel(
        y_labels[i],
        fontsize = 12,
        fontweight="bold"
    )
    ax[i].yaxis.set_major_locator(y_major_locators[i])
    ax[i].yaxis.set_minor_locator(y_minor_locators[i])

# Límites eje Y
ax[2].set_ylim([0, data.tail(cotizaciones_a_mostrar).riesgo_pais.max() * 1.1])

# Le damos estilo al grid del fondo
grid_estilo = {"color": "silver", "linestyle": "--", "linewidth": 0.5}
for axis in ax:
    axis.grid(**grid_estilo)

ax[0].legend(prop={'size': 8}, shadow=True)
ax[1].legend(prop={'size': 8}, shadow=True)
   
# Guardamos el gráfico como imagen .jpg para enviarla por mail
fig.tight_layout(pad=1)
graficos_jpg = "Gráficos Tipos de Cambios y Riesgo País.jpg"
plt.savefig(graficos_jpg)

In [ ]:
# Crear un subplot con tres gráficos verticales
fig, axes = plt.subplots(3, 1, figsize=(10, 14), sharex=True)

fig.suptitle(
    f"Evolución de la Inflación Argentina en los últimos {cantidad_filas} meses",
    fontweight="bold",
    fontsize=18
)

# Gráfico inflación mensual NO ACUMULADA
sns.lineplot(
    x="Fecha",
    y="Inflación",
    data=inflacion_df,
    color="darkred",
    marker='o',
    markersize=8,
    ax=axes[0]
)

# Configurar los ticks del eje X para mostrar cada 3 meses
axes[0].set_xticks(range(0, cantidad_filas, 3))
axes[0].set_xticklabels(inflacion_df.Fecha[::3], rotation=45)
axes[0].set_xlim(0, cantidad_filas - 1)

axes[0].set_title(
    "Evolución de la Inflación Mensual (no acumulada)",
    fontweight="bold",
    fontsize=12
)
axes[0].set_ylabel(
    "Inflación Mensual",
    fontweight="bold",
    fontsize=12
)
axes[0].set_xlabel("")

axes[0].yaxis.set_major_formatter(FuncFormatter("{:,.0f} %".format))
axes[0].grid(True)
sns.despine()

# Gráfico inflación mensual ACUMULADA
sns.lineplot(
    x="Fecha",
    y="Inflación Agregada",
    data=inflacion_df,
    color="darkred",
    linewidth=2,
    legend=False,
    ax=axes[1]
)

# Configurar los ticks del eje X para mostrar cada 6 meses
axes[1].set_xticks(range(0, cantidad_filas, 6))
axes[1].set_xticklabels(inflacion_df.Fecha[::6], rotation=45)
axes[1].set_xlim(0, cantidad_filas - 1)

axes[1].set_title(
    "Inflación Acumulada",
    fontweight="bold",
    fontsize=12
)
axes[1].set_ylabel(
    "Inflación Acumulada",
    fontweight="bold",
    fontsize=12
)

axes[1].yaxis.set_major_formatter(FuncFormatter("{:,.0f} %".format))
axes[1].grid(True)
sns.despine()

# Gráfico inflación mensual ACUMULADA logarítmica
sns.lineplot(
    x="Fecha",
    y="Inflación Agregada",
    data=inflacion_df,
    color="darkred",
    linewidth=2,
    ax=axes[2]
).set_yscale("log")  # Escala logarítmica en el eje Y

# Configurar los ticks del eje X para mostrar cada 3 meses
axes[2].set_xticks(range(0, cantidad_filas, 3))
axes[2].set_xticklabels(inflacion_df.Fecha[::3], rotation=45)
axes[2].set_xlim(0, cantidad_filas - 1)
axes[2].set_xlabel("")

axes[2].set_title(
    "Inflación Acumulada (escala logarítmica)",
    fontweight="bold",
    fontsize=14
)
axes[2].set_ylabel(
    "Inflación Acumulada",
    fontweight="bold",
    fontsize=12
)

axes[2].yaxis.set_major_formatter(FuncFormatter("{:,.0f} %".format))
axes[2].grid(True)
sns.despine()

# Ajustar el espaciado entre los gráficos
plt.tight_layout()

# Guardamos el gráfico como imagen .jpg para enviarla por mail
fig.tight_layout(pad=1)

inflacion_jpg = "Gráficos Inflación.jpg"
plt.savefig(inflacion_jpg)

# Mostrar el gráfico
plt.show()

In [ ]:
inflacion_df["Fecha"] = pd.to_datetime(inflacion_df["Fecha"], format="%m/%y")

data["Fecha"] = pd.to_datetime(data["Fecha"], format="%d/%m/%y")

variacion_acumulada = data.merge(inflacion_df[["Fecha","Inflación"]], on=["Fecha"], how="outer")

#Ordenamos por fecha ya que luego del merge, pone al final las fechas sin cotización pero con inflación
variacion_acumulada.sort_values("Fecha", inplace=True)

# Filtramos por el año actual
current_year = pd.Timestamp.today().year

#Damos vuelta el dataset
variacion_acumulada = variacion_acumulada[variacion_acumulada["Fecha"].dt.year == current_year]

variacion_acumulada.loc[:, "Fecha"] = pd.to_datetime(variacion_acumulada["Fecha"], format="%d-%m-%y")

#Creamos las columnas acumulativas
columns_to_cumulate = ['Variación Solidario', 'Variación TCV Blue', 'Inflación']
for column in columns_to_cumulate:
    if column == 'Inflación':
        variacion_acumulada[f'{column} Acumulada'] = ((1 + variacion_acumulada[column] / 100).cumprod()-1)*100
    else:
        variacion_acumulada[f'{column} Acumulada'] = ((1 + variacion_acumulada[column]).cumprod()-1)*100

        
cantidad_filas_variacion = len(variacion_acumulada) 
fig, ax = plt.subplots(figsize=(10, 5))

# Gráfico de cotizaciones oficiales
palette = sns.color_palette("deep")
ax2_columns = [
    "Variación Solidario Acumulada",
    "Variación TCV Blue Acumulada",
    "Inflación Acumulada"
]
ax2_labels = [
    "Δ Solidario Acumulada",
    "Δ TCV Blue Acumulada",
    "Inflación Acumulada"
]

for i, column in enumerate(ax2_columns):

    if column == "Inflación Acumulada":
        sns.lineplot(
            x="Fecha",
            y=column,
            data=variacion_acumulada,
            label=ax2_labels[i],
            color=palette[i],
            linewidth=2,
            marker="D",
            markersize=7
        )
    else:
        sns.lineplot(
            x="Fecha",
            y=column,
            data=variacion_acumulada,
            label=ax2_columns[i],
            color=palette[i],
            linewidth=2
        )

    # Etiqueta de datos para cada línea, se requieren diferentes frecuencias
    bbox = dict(facecolor=palette[i], alpha=1, edgecolor='black', boxstyle='square,pad=0.2')
    label_frequency = 25 if column != "Inflación Acumulada" else 1
    for idx, row in variacion_acumulada.iterrows():
        if not pd.isna(row[column]):
            # Etiquetar según frecuencia correspondiente
            if idx % label_frequency == 0:
                plt.text(
                    row['Fecha'],
                    row[column]*1.05,
                    f'{row[column]:,.0f}%',
                    fontsize=9,
                    color="white",
                    fontweight="bold",
                    ha="center",
                    va="center",
                    bbox=bbox
                )

            # Etiqueta del valor máximo por línea/columna
            bbox_max = dict(facecolor="lightgray", alpha=1, edgecolor="black", boxstyle='square,pad=0.2')
            if row[column] == variacion_acumulada[column].max() and idx == variacion_acumulada[column].idxmax():
                plt.text(
                    row['Fecha'],
                    row[column]*1.05,
                    f'{row[column]:,.0f}%',
                    fontsize=10,
                    color="red",
                    fontweight="bold",
                    ha="center",
                    va="center",
                    bbox=bbox_max
                )

ax.set_title("Variaciones acumuladas durante el año", fontsize=14, fontweight="bold")

# Formatear eje de X
import matplotlib.dates as mdates
ax.xaxis.set_major_formatter(mdates.DateFormatter("%B"))
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))  # Mostrar un tick al mes

# Formatear eje de Y
ax.yaxis.set_major_formatter(FuncFormatter("{:,.0f} %".format))
ax.set_xlabel("")
ax.set_ylabel("Variación Acumulada en %", fontsize = 12, fontweight="bold")

plt.grid(True)
sns.despine()

variaciones_jpg = "Variaciones.jpg"
plt.savefig(variaciones_jpg)

plt.legend(prop={'size': 8}, shadow=True)
plt.show()

## 📮 Mailing automático

In [ ]:
# Declaración de variables, para que funcione tu contraseña de gmail, tenés que tener una contraseña para app
# si no tenés, se crea en esta página myaccount.google.com
email_sender = input("Tipeá tu usuario de gmail:")
email_password = getpass("Tipeá tu contraseña:")

email_receiver = [
    "martinezmauroezequiel@gmail.com",
    "siquilacamila@gmail.com",
    "pedraza.marcelo@gmail.com",
    "gabrielasiquila@gmail.com",
    "maximo.garcia@grupohst.com",
    "garciiamaximo@gmail.com",
    "fretesricardo20@gmail.com",
    "danielignacio605@gmail.com",
    "alannicolassauer@hotmail.com",
    "djacue@gmail.com",
    "rochita.sp@hotmail.com",
    "renzo.deleo@gmail.com",
    "m.cardozo@wega.com.ar",
    "facundoezequiel_andrada@hotmail.com",
    "gonzaloeorellana@gmail.com",
    "mariano_danze@hotmail.com",
    "duartecami28@gmail.com",
    "sscunico@gmail.com",
    "msguerrieri@bybglobalcorp.com",
    "jlezcano@bybglobalcorp.com",
    "eveiguelo@yahoo.com.ar",
    "juanmeils@hotmail.com",
    "agalean7@gmail.com",
    "apavlovsky@bybglobalcorp.com",
    "bibybsa@gmail.com",
    "mmartinez@bybglobalcorp.com",
    "rgonzalohernandez095@gmail.com"
]

# email_receiver = email_sender

# Tabular los días a mostrar del dataframe para que se vea en el cuerpo del mail
cotizaciones = [
    "Fecha",
    "Solidario",
    "tccblue",
    "tcvblue",
    "MEP",
    "tccBillete",
    "tcvBillete",
    "tccDivisas",
    "tcvDivisas",
    "tccEuros",
    "tcvEuros",
    "FEDtea",
    "BCRAtea",
    "RiesgoPaís"
]

# Valores correspondientes a los últimos días, 14 columnas
valores = df.iloc[:cotizaciones_a_mostrar, :14].values.tolist()
tabla_cotizaciones = tabulate(
    valores,
    headers=cotizaciones,
    tablefmt = "html",
    numalign="center",
    floatfmt=".2f"
)

# Encabezados para tabular últimos 10 días del df y ponerlo en el cuerpo
variaciones = [
    "Solidario/TCV_Blue",
    "MEP/TCV_Blue",
    "Euro_Blue/Dólar_Blue",
    "Variación_Solidario",
    "Variación_Blue",
    "Variación_Euro"
]
# Valores correspondientes a los últimos días, 14 columnas
valores = df.iloc[:cotizaciones_a_mostrar, 14:].values.tolist()
tabla_variaciones = tabulate(
    valores,
    headers=variaciones,
    tablefmt="html",
    numalign="center",
    floatfmt=".2%"
)

# Cuerpo del texto con código html
html = f"""<!DOCTYPE html>
<html lang="es">
   <head>
      <meta charset="UTF-8">
      <meta http-equiv="X-UA-Compatible" content="IE=edge">
      <meta name="viewport" content="width=device-width, initial-scale=1.0">
   </head>
   <body>
      <p> 📅 <b><u>Resumen diario</u></b> ℹ️<br></p>
      <p>
         <b>Costo comprar USD 200 blue 💰:</b> $ {(float(df["TCV_Blue"].iloc[0] * 200)):,.2f} <br>
         <b>Costo comprar USD 200 oficiales con impuestos:</b> $ {(float(df["Solidario"].iloc[0] * 200)):,.2f}<br>
         <b>La diferencia es de:</b> $ {(float(df["TCV_Blue"].iloc[0] * 200 - df["Solidario"].iloc[0] * 200)):,.2f}<br><br>
         <b>La brecha entre solidario y blue es:</b> {float((1 - df["Solidario"].iloc[0] / df["TCV_Blue"].iloc[0]) * 100):.2f}%<br>
         <b>El dólar solidario varió:</b> {float(((df["Solidario"].iloc[0] / df["Solidario"].iloc[1] - 1) * 100)):.2f}%<br>
         <b>El dólar blue varió 💵:</b> {float(((df["TCV_Blue"].iloc[0] / df["TCV_Blue"].iloc[1] - 1) * 100)):.2f}%<br>
         <b>El dólar divisas varió:</b> {float(((df["TCV_Divisas"].iloc[0] / df["TCV_Divisas"].iloc[1] - 1) * 100)):.2f}%<br><br>
         <b>El euro varió un 💶:</b> {float(((df["TCV_Euro"].iloc[0] / df["TCV_Euro"].iloc[1] - 1) * 100)):.2f}%<br>
         <b>Entre euro blue y dólar blue hay una brecha de:</b> {(1-df["TCV_Blue"].iloc[0]/df["TCV_Euro"].iloc[0])*100:.2f}%<br><br>
         <b>El valor del dólar oficial futuro según paridad de tasas de interés es:</b> $ {fwd_oficial:,.2f}<br>
         <b>El valor del dólar blue futuro según paridad de tasas de interés es 💸:</b> ${fwd_blue:,.2f}<br>
         <b>El riesgo país hoy es de ☣️:</b> {(float(df["riesgo_pais"].iloc[0])):,.0f}, es decir que Argentina debe pagar un {(float(df["riesgo_pais"].iloc[0] / 100)):.2f}% de sobretasa respecto a un bono del tesoro estadounidense.
      </p>
      <img src='cid:image1'>
      <img src='cid:image2'>
      <img src='cid:image3'>
      <p>Las siguientes tablas muestran las últimas {cotizaciones_a_mostrar} cotizaciones de dólares, euro, riesgo país y TEA del BCRA 🇦🇷 y TEA de la FED 🇺🇸.<br>
      Analizalas mientras te cebás unos mates 🧉:<br>
      </p>
      {tabla_cotizaciones}<br>
      {tabla_variaciones}<br>
      <p><b><u>📚Las fuentes consultadas fueron:</u>
      <b><br>Cotizaciones oficiales: </b>{web_bna}
      <b><br>Cotizaciones paralelas y Solidario: </b>{web_dolarhoy}
      <b><br>Cotización Dólar MEP: </b>{web_ambito_mep}
      <b><br>Riesgo País: </b>{web_ambito_riesgo_pais}
      <b><br>Cotización Euro Blue: </b>{web_perfil}
      <b><br>Tasa Efectiva Anual EEUU: </b>{web_fed}
      <b><br>Tasa Efectiva Anual e Inflación Argentina: </b>{web_bcra}<br>
      </p>
      <p>
         🤓 Muchas gracias por leer,<br>
         🤑 Ojalá te sirva,<br><br>
      <p> 
         👨🏽‍💼 En Linkedin encontrá mi historial laboral, académico y mis aportes en datos y comercio internacional: <b>https://linkedin.com/in/mauroemartinez/</b> <br>
         👨🏽‍💻 Encontrá en GitHub mis proyectos. ¡Incluso este!: <b>https://github.com/mauroemartinez/</b><br><br>
         🤖<i> Este es un mail automático creado por <b>Mauro E. Martinez</b>, cuya ejecución duró ⏰ {time.perf_counter() - comienzo:.2f} segundos. Escriba a martinezmauroezequiel@gmail.com ante cualquier duda, feedback, desuscripción u ofertas de colaboración.
      </p>
      </p>  
   </body>
</html>"""

# Crear el objeto MIMEMultipart y asignarle los campos
# A los receptores los ubico en Cco para que no se vean entre sí
em = MIMEMultipart('related')
em['From'] = email_sender
em["To"] = email_sender
em['Cco'] = ', '.join(email_receiver)
em["Subject"] = f"📈 Reporte automático de tipos de cambio  💱 - {dt.datetime.today():%d-%m-%Y} 📧"

# Crear el cuerpo del mensaje como un objeto MIMEText y adjuntarlo al objeto MIMEMultipart
body = MIMEText(html, 'html')
em.attach(body)

# Crear una lista con los nombres de los archivos a adjuntar
archivos_adjuntos = [graficos_jpg, inflacion_jpg, variaciones_jpg]
# Iterar a través de la lista y adjuntar cada uno
for archivo in archivos_adjuntos:
    with open(archivo, 'rb') as fp:
        msgImage = MIMEImage(fp.read())
    # Definir ID de la imagen, tal como en el html, adjuntar
    msgImage.add_header('Content-ID', f'<image{archivos_adjuntos.index(archivo) + 1}>')
    msgImage.add_header('Content-Disposition', 'attachment', filename=archivo)
    em.attach(msgImage)

# # Adjuntar .csv
# bd_filename = 'Seguimiento Dólar Blue.csv'
# with open(bd_filename, 'r') as fp:
#     bd_data = fp.read()
# bd = MIMEText(bd_data, 'csv')
# bd.add_header('Content-Disposition', 'attachment', filename=bd_filename)
# em.attach(bd)

# Crear un contexto SSL y establecer una conexión SMTP segura con el servidor de correo
context = ssl.create_default_context()
with smtplib.SMTP_SSL(
    "smtp.gmail.com",
    465,
    context=context) as smtp:
    smtp.login(
        email_sender,
        email_password
        )
    smtp.sendmail(
        email_sender,
        email_receiver,
        em.as_string()
        )

# # para outlook
# context = ssl.create_default_context()
# with smtplib.SMTP("smtp-mail.outlook.com", 587) as server:
#     server.ehlo()
#     server.starttls(context=context)
#     server.ehlo()
#     server.login(email_sender, email_password)
#     server.sendmail(email_sender, email_receiver, em.as_string())

print(f"La duración de la ejecución total fue de: {time.perf_counter() - comienzo:.2f} segundos")